## Note:
* Producer is Distributed Scrapy Crawler. store at `self.paragraphs` for publisher.
* Publisher is background thread running in scrapy. pop from `self.paragraphs`.
* Consumer is ec2 server

<br>

* **Start Threads at scrapy __init__**

* **join threads at the end of crawling**


In [1]:
import csv

# Read first line of csv file
file_path = "/home/anon/Downloads/crawled_nepali_news_dataset.csv"

# Open the file in read mode
with open(file_path, 'r') as file:
    reader = csv.reader(file)
    # Read the first five lines
    for i in range(5):
        line = next(reader)
        print(line)

['paragraph', 'parent_url', 'page_title', 'is_nepali_confidence']
['प्रदेश सरकार र निजी क्षेत्रको सहकार्यमा पहाडी तथा हिमाली क्षेत्रमा मनोरञ्जनात्मक तथा साहसिक पर्यटनको लागि हिलस्टेशनहरू विकास गर्न आवश्यक छ भन्दै उनले सांस्कृतिक, धार्मिक, साहसिक, कृषि, स्वास्थ्य तथा खेल पर्यटक आकर्षित गर्दै यस क्षेत्रको मौलिक संस्कृति संरक्षणमा महोत्सवले सहयोग गर्ने विश्वास व्यक्त गरे ।\xa0', 'https://hamrakura.com/news-details/161504/2023-12-27', 'लोकतन्त्रको आन्दोलन उठाउँदाकै आस्थाबाट निर्देशित छु – राष्ट्रपति पौडेल', '-1914.427728056908']
['त्रिवेणी बाहेक अन्य पालिकाबाट कुन-कुन घर परिवारले रकम पाउने भन्ने विवरण नआइसकेकाले ती पालिकाका लागि रकम निकासा भने हुन सकेको छैन ।', 'https://hamrakura.com/news-details/159820/2023-11-28', 'भूकम्पपीडितको अस्थायी आवासका लागि रकम निकासा', '-800.0689898729324']
['निर्वाचित मण्डलले निर्वाचनका सबै प्रक्रिया अघि बढाएपनी सहमतिका लागि शीर्ष नेताहरूले समय मागेकाले निर्वाचन कमिटीले समय दिएको थियो । निर्वाचन कमिटीका संयोजक जगत बहादुर रोकायाले बताए ।', 'https://hamrakura.com

In [1]:
import csv
import dotenv
import json
import os
import redis
import threading
import time
dotenv.load_dotenv()



# Connect to your Redis server
redis_client = redis.Redis(
    host=os.environ.get('REDIS_HOST', 'localhost'),
    port = int(os.environ.get('REDIS_PORT', 6379)),
    password=os.environ.get('REDIS_PASSWORD', None),
)




def pop_from_redis():
    # print('call gari rako muji')
    lists_to_pop = ['crawled_data']       # , 'crawled', 'to_crawl']
    popped_data = []
    for list_name in lists_to_pop:
        # print(f'list_name:{list_name}')
        while redis_client.llen(list_name) > 0:
            # print(f'list_name:{list_name}')
            popped_data.append(json.loads(redis_client.rpop(list_name)))
    # print("return vayo muji")
    return popped_data

def push_to_redis(list_name, data):
    redis_client.lpush(list_name, data)

def save_to_csv(data, csv_file_path="crawled_data.csv"):
    field_names = ['paragraph', 'parent_url', 'page_title', 'is_nepali_confidence']
    file_exists = os.path.exists(csv_file_path)
    print(f'file_exists: {file_exists}')
    # Open the CSV file in append mode
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
        # Create a CSV writer object
        csv_writer = csv.DictWriter(csvfile, fieldnames=field_names)

        # If the file doesn't exist, write the header
        if not file_exists:
            csv_writer.writeheader()

        # Append the new data
        csv_writer.writerows(data)

def load_from_csv(csv_file_path="crawled_data.csv"):
    data = []
    # Open the CSV file in read mode
    with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
        # Create a CSV reader object
        csv_reader = csv.DictReader(csvfile)

        # Read and print each row of data
        for row in csv_reader:
            data.append(row)
            # print(row)
    return data

def producer():
    import random
    print('producer')
    data_paragraphs_copy = [{'paragraph': 'प्रदेश सरकार र निजी क्षेत्रको सहकार्यमा पहाडी तथा हिमाली क्षेत्रमा मनोरञ्जनात्मक तथा साहसिक पर्यटनको लागि हिलस्टेशनहरू विकास गर्न आवश्यक छ भन्दै उनले सांस्कृतिक, धार्मिक, साहसिक, कृषि, स्वास्थ्य तथा खेल पर्यटक आकर्षित गर्दै यस क्षेत्रको मौलिक संस्कृति संरक्षणमा महोत्सवले सहयोग गर्ने विश्वास व्यक्त गरे ।\xa0', 'parent_url': 'https://hamrakura.com/news-details/161504/2023-12-27', 'page_title': 'लोकतन्त्रको आन्दोलन उठाउँदाकै आस्थाबाट निर्देशित छु – राष्ट्रपति पौडेल', 'is_nepali_confidence':'-1914.427728056908'},
        {'paragraph': 'त्रिवेणी बाहेक अन्य पालिकाबाट कुन-कुन घर परिवारले रकम पाउने भन्ने विवरण नआइसकेकाले ती पालिकाका लागि रकम निकासा भने हुन सकेको छैन ।', 'parent_url': 'https://hamrakura.com/news-details/159820/2023-11-28', 'page_title': 'भूकम्पपीडितको अस्थायी आवासका लागि रकम निकासा', 'is_nepali_confidence':'-800.0689898729324'},
        {'paragraph': 'निर्वाचित मण्डलले निर्वाचनका सबै प्रक्रिया अघि बढाएपनी सहमतिका लागि शीर्ष नेताहरूले समय मागेकाले निर्वाचन कमिटीले समय दिएको थियो । निर्वाचन कमिटीका संयोजक जगत बहादुर रोकायाले बताए ।','parent_url': 'https://hamrakura.com/news-details/160003/2023-12-01', 'page_title': 'अध्यक्ष मण्डलले घोषणा गरे जिल्ला कमिटी, टिके प्रथा चलाएको रावल पक्षको आरोप', 'is_nepali_confidence':'-1128.5258438587189'},
        {'paragraph': 'अहिलेसम्म एनसेलले शेयर किनबेच गरेको सम्बन्धमा नेपाल दूरसञ्चार प्राधिकरणले गरेको काम कारबाहीको सम्बन्धमा जानकारी माग्ने पत्र लेख्ने', 'parent_url':'https://hamrakura.com/news-details/161068/2023-12-19', 'page_title': 'एनसेलले राज्यलाई तिर्नुपर्ने कर असुल उपर गर्न सरकारलाई समितिको निर्देशन [भिडियो]', 'is_nepali_confidence':'-800.2218471765518'}
        ]
    while True:
        # add n items to the list
        n_items = random.randint(0, 3)
        data_paragraphs.extend(data_paragraphs_copy[:n_items])
        print(f'produced: {n_items}')

        # sleep randomly between 0 and 5 seconds
        time.sleep(random.randint(0, 5))

def publisher():
    print('published')
    while True:
        
        if data_paragraphs:
            pushed = 0
            for paragraph in data_paragraphs:
                push_to_redis('crawled_data', json.dumps(data_paragraphs.pop()))
                pushed += 1
            print(f'---published: {pushed}---')
        else:
            time.sleep(5)  # sleep for a while before producing more items
        # time.sleep(1)  # sleep for a while before producing more items

def consumer():
    print('consumer')
    pulled = 0
    while True:
        # print('consumer')
        paragraphs = pop_from_redis()
        
        if paragraphs:
            save_to_csv(paragraphs)
            pulled += len(paragraphs)
            print(f'======consumed: {len(paragraphs)}')     #\n\n current_count:{redis_client.llen("paragraphs")}')
            # print(f'len(paragraphs): {len(paragraphs)} \n\n paragraphs:{paragraphs}')
        else:
            print('------------------')   # No Data
            time.sleep(1)  # sleep for a while before consuming more items

data_paragraphs = []

# Create producer and consumer threads
producer_thread = threading.Thread(target=producer)
publisher_thread = threading.Thread(target=publisher)
consumer_thread = threading.Thread(target=consumer)


# Start the threads
producer_thread.start()
# consumer_thread.start()
publisher_thread.start()

# Wait for both threads to finish
producer_thread.join()
publisher_thread.join()
# consumer_thread.join()


producer
produced: 3
produced: 0
published
---published: 2---
---published: 1---
produced: 2
produced: 2
produced: 1
---published: 3---
---published: 1---
---published: 1---
produced: 1
produced: 3
produced: 3
produced: 3
---published: 5---
---published: 3---
---published: 1---
---published: 1---
produced: 1
produced: 1
---published: 1---
---published: 1---
produced: 2
produced: 0
produced: 3
produced: 1
produced: 2
produced: 0
produced: 0
---published: 4---
---published: 2---
---published: 1---
---published: 1---
produced: 0
produced: 2
---published: 1---
---published: 1---
produced: 3
produced: 3
produced: 2
produced: 1
---published: 5---
---published: 2---
---published: 1---
---published: 1---
produced: 3
produced: 1
produced: 2
---published: 3---
---published: 2---
---published: 1---
produced: 2
produced: 0
produced: 1
---published: 2---
---published: 1---
produced: 1
---published: 1---
produced: 2
produced: 0
---published: 1---
---published: 1---
produced: 3
---published: 2---
---

In [23]:
import csv
import dotenv
import json
import os
import redis
import threading
import time
dotenv.load_dotenv()



# Connect to your Redis server
redis_client = redis.Redis(
    host=os.environ.get('REDIS_HOST', 'localhost'),
    port = int(os.environ.get('REDIS_PORT', 6379)),
    password=os.environ.get('REDIS_PASSWORD', None),
)
redis_client.llen('crawled_data')
redis_client.llen('quotes')
redis_client.llen('to_visit')
# datas = []
# while redis_client.llen('quotes') > 0:
#     datas.append(json.loads(redis_client.rpop('quotes')))
#     print()
    # time.sleep(1)


8

In [29]:
datas =  []
while redis_client.llen('to_visit') > 0:
    datas.append(redis_client.rpop('to_visit').decode('utf-8'))
datas

[]

## Consumer only: 'Crawled_data' and 'other_data'

In [ ]:
data_paragraphs_copy = [{'paragraph': 'प्रदेश सरकार र निजी क्षेत्रको सहकार्यमा पहाडी तथा हिमाली क्षेत्रमा मनोरञ्जनात्मक तथा साहसिक पर्यटनको लागि हिलस्टेशनहरू विकास गर्न आवश्यक छ भन्दै उनले सांस्कृतिक, धार्मिक, साहसिक, कृषि, स्वास्थ्य तथा खेल पर्यटक आकर्षित गर्दै यस क्षेत्रको मौलिक संस्कृति संरक्षणमा महोत्सवले सहयोग गर्ने विश्वास व्यक्त गरे ।\xa0', 'parent_url': 'https://hamrakura.com/news-details/161504/2023-12-27', 'page_title': 'लोकतन्त्रको आन्दोलन उठाउँदाकै आस्थाबाट निर्देशित छु – राष्ट्रपति पौडेल', 'is_nepali_confidence':'-1914.427728056908'},
        {'paragraph': 'त्रिवेणी बाहेक अन्य पालिकाबाट कुन-कुन घर परिवारले रकम पाउने भन्ने विवरण नआइसकेकाले ती पालिकाका लागि रकम निकासा भने हुन सकेको छैन ।', 'parent_url': 'https://hamrakura.com/news-details/159820/2023-11-28', 'page_title': 'भूकम्पपीडितको अस्थायी आवासका लागि रकम निकासा', 'is_nepali_confidence':'-800.0689898729324'},
        {'paragraph': 'निर्वाचित मण्डलले निर्वाचनका सबै प्रक्रिया अघि बढाएपनी सहमतिका लागि शीर्ष नेताहरूले समय मागेकाले निर्वाचन कमिटीले समय दिएको थियो । निर्वाचन कमिटीका संयोजक जगत बहादुर रोकायाले बताए ।','parent_url': 'https://hamrakura.com/news-details/160003/2023-12-01', 'page_title': 'अध्यक्ष मण्डलले घोषणा गरे जिल्ला कमिटी, टिके प्रथा चलाएको रावल पक्षको आरोप', 'is_nepali_confidence':'-1128.5258438587189'},
        {'paragraph': 'अहिलेसम्म एनसेलले शेयर किनबेच गरेको सम्बन्धमा नेपाल दूरसञ्चार प्राधिकरणले गरेको काम कारबाहीको सम्बन्धमा जानकारी माग्ने पत्र लेख्ने', 'parent_url':'https://hamrakura.com/news-details/161068/2023-12-19', 'page_title': 'एनसेलले राज्यलाई तिर्नुपर्ने कर असुल उपर गर्न सरकारलाई समितिको निर्देशन [भिडियो]', 'is_nepali_confidence':'-800.2218471765518'}
        ]

In [30]:
import csv
import dotenv
import json
import os
import redis
import threading
import time
dotenv.load_dotenv()


def pop_from_redis():
    # print('call gari rako muji')
    lists_to_pop = ['crawled_data', 'other_data']       # , 'crawled', 'to_crawl']
    popped_data = {}
    for list_name in lists_to_pop:
        if list_name not in popped_data:
            popped_data[list_name] = []
        # print(f'list_name:{list_name}')
        while redis_client.llen(list_name) > 0:
            # print(f'list_name:{list_name}')
            popped_data[list_name].append(json.loads(redis_client.rpop(list_name)))
    # print("return vayo muji")
    return popped_data

def push_to_redis(list_name, data):
    redis_client.lpush(list_name, data)


# Connect to your Redis server
redis_client = redis.Redis(
    host=os.environ.get('REDIS_HOST', 'localhost'),
    port = int(os.environ.get('REDIS_PORT', 6379)),
    password=os.environ.get('REDIS_PASSWORD', None),
)



def save_to_csv(data, data_type="crawled_data"):
    for key, data_items in data.items():
        csv_file_path = key + ".csv"
        if data_items:
            # field_names = ['paragraph', 'parent_url', 'page_title', 'is_nepali_confidence']
            field_names = data_items[0].keys()
            file_exists = os.path.exists(csv_file_path)
            print(f'file_exists: {file_exists}')
            # Open the CSV file in append mode
            with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
                # Create a CSV writer object
                csv_writer = csv.DictWriter(csvfile, fieldnames=field_names)

                # If the file doesn't exist, write the header
                if not file_exists:
                    csv_writer.writeheader()

                # Append the new data
                csv_writer.writerows(data_items)

def load_from_csv(csv_file_path="crawled_data.csv"):
    data = []
    # Open the CSV file in read mode
    with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
        # Create a CSV reader object
        csv_reader = csv.DictReader(csvfile)

        # Read and print each row of data
        for row in csv_reader:
            data.append(row)
            # print(row)
    return data

def producer():
    import random
    print('producer')
    data_paragraphs_copy = [{'paragraph': 'प्रदेश सरकार र निजी क्षेत्रको सहकार्यमा पहाडी तथा हिमाली क्षेत्रमा मनोरञ्जनात्मक तथा साहसिक पर्यटनको लागि हिलस्टेशनहरू विकास गर्न आवश्यक छ भन्दै उनले सांस्कृतिक, धार्मिक, साहसिक, कृषि, स्वास्थ्य तथा खेल पर्यटक आकर्षित गर्दै यस क्षेत्रको मौलिक संस्कृति संरक्षणमा महोत्सवले सहयोग गर्ने विश्वास व्यक्त गरे ।\xa0', 'parent_url': 'https://hamrakura.com/news-details/161504/2023-12-27', 'page_title': 'लोकतन्त्रको आन्दोलन उठाउँदाकै आस्थाबाट निर्देशित छु – राष्ट्रपति पौडेल', 'is_nepali_confidence':'-1914.427728056908'},
        {'paragraph': 'त्रिवेणी बाहेक अन्य पालिकाबाट कुन-कुन घर परिवारले रकम पाउने भन्ने विवरण नआइसकेकाले ती पालिकाका लागि रकम निकासा भने हुन सकेको छैन ।', 'parent_url': 'https://hamrakura.com/news-details/159820/2023-11-28', 'page_title': 'भूकम्पपीडितको अस्थायी आवासका लागि रकम निकासा', 'is_nepali_confidence':'-800.0689898729324'},
        {'paragraph': 'निर्वाचित मण्डलले निर्वाचनका सबै प्रक्रिया अघि बढाएपनी सहमतिका लागि शीर्ष नेताहरूले समय मागेकाले निर्वाचन कमिटीले समय दिएको थियो । निर्वाचन कमिटीका संयोजक जगत बहादुर रोकायाले बताए ।','parent_url': 'https://hamrakura.com/news-details/160003/2023-12-01', 'page_title': 'अध्यक्ष मण्डलले घोषणा गरे जिल्ला कमिटी, टिके प्रथा चलाएको रावल पक्षको आरोप', 'is_nepali_confidence':'-1128.5258438587189'},
        {'paragraph': 'अहिलेसम्म एनसेलले शेयर किनबेच गरेको सम्बन्धमा नेपाल दूरसञ्चार प्राधिकरणले गरेको काम कारबाहीको सम्बन्धमा जानकारी माग्ने पत्र लेख्ने', 'parent_url':'https://hamrakura.com/news-details/161068/2023-12-19', 'page_title': 'एनसेलले राज्यलाई तिर्नुपर्ने कर असुल उपर गर्न सरकारलाई समितिको निर्देशन [भिडियो]', 'is_nepali_confidence':'-800.2218471765518'}
        ]
    while True:
        # add n items to the list
        n_items = random.randint(0, 3)
        data_paragraphs.extend(data_paragraphs_copy[:n_items])
        print(f'produced: {n_items}')

        # sleep randomly between 0 and 5 seconds
        time.sleep(random.randint(0, 5))

def publisher():
    print('published')
    while True:
        
        if data_paragraphs:
            pushed = 0
            for paragraph in data_paragraphs:
                push_to_redis('crawled_data', json.dumps(data_paragraphs.pop()))
                pushed += 1
            print(f'---published: {pushed}---')
        else:
            time.sleep(5)  # sleep for a while before producing more items
        # time.sleep(1)  # sleep for a while before producing more items

def consumer():
    print('consumer')
    pulled = 0
    while True:
        # print('consumer')
        paragraphs = pop_from_redis()
        
        if paragraphs:
            save_to_csv(paragraphs)
            pulled += len(paragraphs)
            print(f'======consumed: {len(paragraphs)}')     #\n\n current_count:{redis_client.llen("paragraphs")}')
            # print(f'len(paragraphs): {len(paragraphs)} \n\n paragraphs:{paragraphs}')
        else:
            print('------------------')   # No Data
            time.sleep(1)  # sleep for a while before consuming more items

data_paragraphs = []

# # Create producer and consumer threads
# producer_thread = threading.Thread(target=producer)
# publisher_thread = threading.Thread(target=publisher)
# consumer_thread = threading.Thread(target=consumer)


# # Start the threads
# producer_thread.start()
# # consumer_thread.start()
# publisher_thread.start()

# # Wait for both threads to finish
# producer_thread.join()
# publisher_thread.join()
# # consumer_thread.join()


In [33]:
import redis

redis_client = redis.Redis(
host='redis-19089.c212.ap-south-1-1.ec2.cloud.redislabs.com',
port=19089,
password='uUzG92fuoNbpDa2922BTIFnpXyPSgIrO')

data_paragraphs = [{'paragraph': 'प्रदेश सरकार र निजी क्षेत्रको सहकार्यमा पहाडी तथा हिमाली क्षेत्रमा मनोरञ्जनात्मक तथा साहसिक पर्यटनको लागि हिलस्टेशनहरू विकास गर्न आवश्यक छ भन्दै उनले सांस्कृतिक, धार्मिक, साहसिक, कृषि, स्वास्थ्य तथा खेल पर्यटक आकर्षित गर्दै यस क्षेत्रको मौलिक संस्कृति संरक्षणमा महोत्सवले सहयोग गर्ने विश्वास व्यक्त गरे ।\xa0', 'parent_url': 'https://hamrakura.com/news-details/161504/2023-12-27', 'page_title': 'लोकतन्त्रको आन्दोलन उठाउँदाकै आस्थाबाट निर्देशित छु – राष्ट्रपति पौडेल', 'is_nepali_confidence':'-1914.427728056908'},
                {'paragraph': 'त्रिवेणी बाहेक अन्य पालिकाबाट कुन-कुन घर परिवारले रकम पाउने भन्ने विवरण नआइसकेकाले ती पालिकाका लागि रकम निकासा भने हुन सकेको छैन ।', 'parent_url': 'https://hamrakura.com/news-details/159820/2023-11-28', 'page_title': 'भूकम्पपीडितको अस्थायी आवासका लागि रकम निकासा', 'is_nepali_confidence':'-800.0689898729324'},
                {'paragraph': 'निर्वाचित मण्डलले निर्वाचनका सबै प्रक्रिया अघि बढाएपनी सहमतिका लागि शीर्ष नेताहरूले समय मागेकाले निर्वाचन कमिटीले समय दिएको थियो । निर्वाचन कमिटीका संयोजक जगत बहादुर रोकायाले बताए ।','parent_url': 'https://hamrakura.com/news-details/160003/2023-12-01', 'page_title': 'अध्यक्ष मण्डलले घोषणा गरे जिल्ला कमिटी, टिके प्रथा चलाएको रावल पक्षको आरोप', 'is_nepali_confidence':'-1128.5258438587189'},
                {'paragraph': 'अहिलेसम्म एनसेलले शेयर किनबेच गरेको सम्बन्धमा नेपाल दूरसञ्चार प्राधिकरणले गरेको काम कारबाहीको सम्बन्धमा जानकारी माग्ने पत्र लेख्ने', 'parent_url':'https://hamrakura.com/news-details/161068/2023-12-19', 'page_title': 'एनसेलले राज्यलाई तिर्नुपर्ने कर असुल उपर गर्न सरकारलाई समितिको निर्देशन [भिडियो]', 'is_nepali_confidence':'-800.2218471765518'}
                ]

for i, paragraph in enumerate(data_paragraphs):
    print(i)
    push_to_redis('crawled_data', json.dumps(data_paragraphs.pop()))


other_data_paragraphs = [{
    'parent_url': "response.url",
    'url': "link.url",
    'text': "link.text",
    'link_type': '<one of ["drive_link", "document", "social_media"]> ',
    'link_description': None
}
]
for i, paragraph in enumerate(other_data_paragraphs):
    print(i)
    push_to_redis('other_data', json.dumps(other_data_paragraphs.pop()))


0
1
0


In [34]:
papragraphs = pop_from_redis()
papragraphs

{'crawled_data': [{'paragraph': 'अहिलेसम्म एनसेलले शेयर किनबेच गरेको सम्बन्धमा नेपाल दूरसञ्चार प्राधिकरणले गरेको काम कारबाहीको सम्बन्धमा जानकारी माग्ने पत्र लेख्ने',
   'parent_url': 'https://hamrakura.com/news-details/161068/2023-12-19',
   'page_title': 'एनसेलले राज्यलाई तिर्नुपर्ने कर असुल उपर गर्न सरकारलाई समितिको निर्देशन [भिडियो]',
   'is_nepali_confidence': '-800.2218471765518'},
  {'paragraph': 'निर्वाचित मण्डलले निर्वाचनका सबै प्रक्रिया अघि बढाएपनी सहमतिका लागि शीर्ष नेताहरूले समय मागेकाले निर्वाचन कमिटीले समय दिएको थियो । निर्वाचन कमिटीका संयोजक जगत बहादुर रोकायाले बताए ।',
   'parent_url': 'https://hamrakura.com/news-details/160003/2023-12-01',
   'page_title': 'अध्यक्ष मण्डलले घोषणा गरे जिल्ला कमिटी, टिके प्रथा चलाएको रावल पक्षको आरोप',
   'is_nepali_confidence': '-1128.5258438587189'},
  {'paragraph': 'अहिलेसम्म एनसेलले शेयर किनबेच गरेको सम्बन्धमा नेपाल दूरसञ्चार प्राधिकरणले गरेको काम कारबाहीको सम्बन्धमा जानकारी माग्ने पत्र लेख्ने',
   'parent_url': 'https://hamrakura.com/

In [35]:
save_to_csv(papragraphs)

file_exists: True
file_exists: True


# SADD and SREM (urls_to_crawl_cleaned_set, url_to_crawl, url_crawling)

In [79]:
redis_client.delete('url_to_crawl')

1

In [91]:
redis_client.sadd('url_to_crawl', json.dumps("https://site_link.url.com.np/what%20ever"))
redis_client.sadd('url_crawling', json.dumps("https://site_link.url.com.np/what%20ever"))

1

In [92]:
# redis_client.smembers('url_to_crawl')
# json.loads(list(redis_client.smembers('url_to_crawl'))[2])
# list(redis_client.smembers('url_to_crawl'))[2].decode('utf-8')

def get_set_data_from_redis(set_names):
    set_data = {}
    for set_name in set_names:
        set_data[set_name] = []
        while redis_client.scard(set_name) > 0:
            set_data[set_name].append(json.loads(redis_client.spop(set_name)))
    return set_data
# for url in list(redis_client.smembers('url_to_crawl')):
#     print(json.loads(url))
#     # print(url)

def save_to_db(url_type, url):
    if url not in db.crawled():
        # url not already crawled
        if url_type == 'url_crawling':
                db.sadd('url_crawling', json.dumps(url))    # add to crawling
                db.srem('url_to_crawl', url)                # remove if exist in to_crawl
        elif url_type == 'url_to_crawl':
            if url not in db.crawling():
                # url not already crawling
                db.sadd('url_to_crawl', json.dumps(url))    # add url in to_crawl

set_names = ['url_to_crawl', 'url_crawling']    # 'urls_to_crawl_cleaned_set'
try:
    for url_type, url in get_set_data_from_redis(set_names).items():
        print(f'url_type: {url_type} \n url: {url}')
        save_to_db(url_type, url)
except Exception as Ex:
    # log the error
    pass


{'url_to_crawl': ['https://site_link.url.com.np/what%20ever'],
 'url_crawling': ['https://site_link.url.com.np/what%20ever']}

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [39]:
redis_client.srem('url_to_crawl', json.dumps("site_link.url"))

0